In [1]:
'''Twitter Sentiment Analysis Using Sklearn and NLTK'''

'Twitter Sentiment Analysis Using Sklearn and NLTK'

In [2]:
# importing some required libraries
import pandas as pd
from nltk.corpus import stopwords
import string
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt



In [3]:
# reading the dataset
columns = ["sentiment", "id", "date", "query", "user_id", "text"]
df = pd.read_csv("trainingTweets.csv", encoding="latin", names=columns) # Give the model training tweets
df.head()





FileNotFoundError: [Errno 2] No such file or directory: 'dataset.csv'

In [ ]:
# replacing the '4' with '1' as positive sentiment
# df["sentiment"] = df["sentiment"].replace(4,1)          # We don't need this for pocsProject.

In [ ]:
# plotting the data using countplot
sns.countplot(x="sentiment",data=df)                     # not necessary to plot total counts...

In [ ]:
#creating a string of positive tweets to analyze the words   # positive means positive for xenophobia
positive_tweets = df[df['sentiment'] == 1]['text'].tolist()
positive_tweets_string = " ".join(positive_tweets)
plt.figure(figsize=(15,15))
plt.imshow(WordCloud().generate(positive_tweets_string))

In [ ]:
# creating the string of negative tweets to analyze the words # negative means negative for xenophobia
negative_tweets = df[df['sentiment'] == 0]['text'].tolist()
negative_tweets_string = " ".join(negative_tweets)
plt.figure(figsize=(15,15))
plt.imshow(WordCloud().generate(negative_tweets_string))

In [ ]:
def tweets_cleaner(tweet):
    # removing the urls from the text
    tweet = re.sub(r'((www.\S+)|(https?://\S+))', r"", tweet)
    #removing the numbers from the text
    tweet = re.sub(r'[0-9]\S+', r'', tweet)
    #removing the tags from the text
    tweet = re.sub(r'(@\S+) | (#\S+)', r'', tweet)
    # removing the punctuation from the text
    tweet_without_punctuation = [char for char in tweet if char not   
                                in string.punctuation]
    # converting the list to string 
    tweet_without_punctuation = "".join(tweet_without_punctuation) 
    # set of stop words 
    stop_words = set(stopwords.words("english"))
    # removing the stop words 
    tweet_without_stopwords = [word for word in  
                              tweet_without_punctuation.split()
                              if word.lower() not in stop_words]
    return tweet_without_stopwords

In [ ]:
# extract the features using count vectorizer
vectorizer = CountVectorizer(analyzer = tweets_cleaner, dtype =     
                            'uint8')
df_countvectorizer = vectorizer.fit_transform(df['text'])
df_countvectorizer.shape

In [ ]:
# splitting the features into train and test
X_train, X_test, y_train, y_test = train_test_split(  
                                                df_countvectorizer,  
                                                df["sentiment"],   # changed from target to sentiment as per "Loading the Dataset"
                                                test_size=0.2,  
                                                 random_state=0)

In [ ]:
# Multinomial Naive Bayes Classifier
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

In [ ]:
# predicting the labels 
pred = classifier.predict(X_test)

print(pred)







In [ ]:
# model accuracy
print(classification_report(y_test, pred))